# Exercise Overview

This is the [exercise](https://github.com/anthropics/courses/blob/master/tool_use/02_your_first_simple_tool.ipynb) from the tool use course from Anthropic.

Given a research topic, the Claude will generate a list of Wikipedia article titles that are relevant to the topic. But since Claude may hallucinate, we need to check the list with the `wikipedia` tool. If an artitle exisits, it will be added to a markdown file.

**To let Claude only use tool when necessary, we need to pass the system prompt to Claude.**


In [2]:
import wikipedia
from anthropic import Anthropic

client = Anthropic()

In [6]:
wikipedia_reading_list_generator_tool = {
    "name": "wikipedia_reading_list_generator",
    "description": "A tool that checks a list of article titles whether they exist in Wikipedia.",
    "input_schema": {
        "type": "object",
        "properties": {
            "article_titles": {
                "type": "array",
                "items": {
                    "type": "string",
                    "description": "The title of a Wikipedia article."
                },
                "description": "The list of article titles that are relevant to the research topic."
            },
        },
        "required": ["article_titles"]
    }
}

In [14]:
def generate_wikipedia_reading_list(research_topic, article_titles):
    wikipedia_articles = []
    for t in article_titles:
        results = wikipedia.search(t)
        try:
            page = wikipedia.page(results[0])
            title = page.title
            url = page.url
            wikipedia_articles.append({"title": title, "url": url})
        except:
            continue
    add_to_research_reading_file(wikipedia_articles, research_topic)

def add_to_research_reading_file(articles, topic):
    with open("output/research_reading.md", "a", encoding="utf-8") as file:
        file.write(f"## {topic} \n")
        for article in articles:
            title = article["title"]
            url = article["url"]
            file.write(f"* [{title}]({url}) \n")
        file.write(f"\n\n")
        
def get_research_help(topic, num_articles=3):
    response = client.messages.create(
        model="claude-3-haiku-20240307",
        system="You have access to tools, but only use them when necessary.  If a tool is not required, respond as normal",
        messages=[{"role": "user", "content":f"generate {num_articles} wikipedia article reading list about {topic}"}],
        max_tokens=400,
        tools=[wikipedia_reading_list_generator_tool]
    )
    print(response)
    if response.stop_reason == "tool_use":
        article_titles = response.content[-1].input["article_titles"]
        print("Claude generates tool use call")
        print("Article titles:", article_titles)
        generate_wikipedia_reading_list(topic, article_titles)
    else:
        print("Claude does not generate tool use call")
        print("Response:", response.content[0].text)
    

In [15]:
get_research_help("Pirates Across The World", 7)

Message(id='msg_01SPivdTg4PiiQA3B9ihEsfs', content=[TextBlock(citations=None, text='Here is a Wikipedia reading list about Pirates Across The World:', type='text'), ToolUseBlock(id='toolu_01Rn7C2fkNXqJQZNKk75Xeeh', input={'article_titles': ['Pirates of the Caribbean', 'History of piracy', 'Golden Age of Piracy', 'Piracy in the Caribbean', 'Piracy in Southeast Asia', 'Piracy in Africa', 'Piracy in South America']}, name='wikipedia_reading_list_generator', type='tool_use')], model='claude-3-haiku-20240307', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=418, output_tokens=125, server_tool_use=None, service_tier='standard'))
Claude generates tool use call
Article titles: ['Pirates of the Caribbean', 'History of piracy', 'Golden Age of Piracy', 'Piracy in the Caribbean', 'Piracy in Southeast Asia', 'Piracy in Africa', 'Piracy in South America']


In [16]:
get_research_help("History of Hawaii", 3)

Message(id='msg_01RxKFxbmuVn9pBrb7GeRwo2', content=[TextBlock(citations=None, text='Okay, let me generate a Wikipedia reading list about the history of Hawaii:', type='text'), ToolUseBlock(id='toolu_01U8NPvAHQ7oSHiNyoSwd833', input={'article_titles': ['History of Hawaii', 'Kingdom of Hawaii', 'Annexation of Hawaii']}, name='wikipedia_reading_list_generator', type='tool_use')], model='claude-3-haiku-20240307', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=416, output_tokens=92, server_tool_use=None, service_tier='standard'))
Claude generates tool use call
Article titles: ['History of Hawaii', 'Kingdom of Hawaii', 'Annexation of Hawaii']


In [17]:
get_research_help("are animals conscious?", 3)

Message(id='msg_01XQz1ymoz35BpFLSxPdKVVJ', content=[TextBlock(citations=None, text='Here is a Wikipedia reading list about whether animals are conscious:', type='text'), ToolUseBlock(id='toolu_01KUW1xsyNKAqCjhSTfuZihJ', input={'article_titles': ['animal consciousness', 'do animals have feelings', 'are animals self-aware']}, name='wikipedia_reading_list_generator', type='tool_use')], model='claude-3-haiku-20240307', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=417, output_tokens=88, server_tool_use=None, service_tier='standard'))
Claude generates tool use call
Article titles: ['animal consciousness', 'do animals have feelings', 'are animals self-aware']


/home/liyuan/miniforge3/envs/deepresearch/lib/python3.12/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/liyuan/miniforge3/envs/deepresearch/lib/python3.12/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
